<a href="https://colab.research.google.com/github/Psousace/Fundos_Imobiliarios/blob/main/Coletando_dados_de_FII_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import numpy as np

## 1. Definindo cabeçalho da requisição

In [ ]:
url = 'https://www.fundsexplorer.com.br/ranking'
headers = {
    'User-Agent':
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36'
        ' (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'
}


In [ ]:
response = requests.get(url, headers=headers)
if response.status_code == 200:
    df = pd.read_html(response.content, encoding='utf-8')[0]

In [ ]:
df.sort_values('Códigodo fundo', inplace=True)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.describe(include='all')

In [ ]:
df['Setor'].unique()

In [ ]:
df.columns

## 2. Limpeza e Formatação de Dados

In [ ]:
df.isna().sum()

### Dados categóricos

In [ ]:
categorical_columns = ['Códigodo fundo','Setor']

In [ ]:
idx = df[df['Setor'].isna()].index
df.drop(idx, inplace=True)

In [ ]:
df[categorical_columns].isna().sum()

In [ ]:
df[categorical_columns] = df[categorical_columns].astype('category')

In [ ]:
df.info()

In [ ]:
df.isna().sum()

### Dados Numéricos

In [ ]:
col_floats = list(df.iloc[:,2:-1].columns)

In [ ]:
## Dados nulos
df[col_floats] = df[col_floats].fillna(value=0)

In [ ]:
df[col_floats]

In [ ]:
df[col_floats].head()

In [ ]:
df[col_floats] = df[col_floats].applymap(lambda x: str(x).replace('R$', '').replace('.0','').replace('.','').replace('%','').replace(',','.'))

In [ ]:
df[col_floats] = df[col_floats].astype('float')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.describe()

- Dados de P/VPA tem atributos infinitos e está em uma escala diferente

In [ ]:
## Check infinity values pandas
df[np.isinf(df[col_floats]).any(1)]

In [ ]:
idx = df[np.isinf(df[col_floats]).any(1)].index
df.drop(idx, inplace=True)

In [ ]:
df['P/VPA'] = df['P/VPA']/100

In [ ]:
df.describe()

### Analisando a média por setor

In [ ]:
df.columns

In [ ]:
indicadores = ['Códigodo fundo',
               'Setor',
               'DY (12M)Acumulado',
               'VacânciaFísica',
               'VacânciaFinanceira',
               'P/VPA',
               'QuantidadeAtivos',
               'Liquidez Diária']

In [ ]:
df_aux = df[indicadores]

In [ ]:
df_aux

In [ ]:
media_setor = df_aux.groupby('Setor').agg(['mean','std'])

In [ ]:
media_setor

In [ ]:
media_setor.loc['Residencial', ('DY (12M)Acumulado', 'mean')]

### Criando uma função com uma estratégia para oportunidades do mercado

In [ ]:
def oportunidade_media_setor(df, setor='Shoppings', label_setor='Setor'):

    media_setor = df_aux.groupby('Setor').agg(['mean','std'])

    df_setor = df[df[label_setor].isin([setor])]

    filter_ = \
            (df_setor['QuantidadeAtivos'] > 5) &\
            (df_setor['Liquidez Diária'] > 5000) &\
            (df_setor['P/VPA'] < 1.0) &\
            (df_setor['DY (12M)Acumulado'] > media_setor.loc[setor, ('DY (12M)Acumulado','mean')])

    print('média do setor Yield: {}'.format(media_setor.loc[setor, ('DY (12M)Acumulado','mean')]))
    print('média do setor p/VPA: {}'.format(media_setor.loc[setor, ('P/VPA','mean')]))
    print('média do setor Ativos: {}'.format(media_setor.loc[setor, ('QuantidadeAtivos','mean')]))

    return df_setor[filter_]

In [ ]:
list(df['Setor'].unique())

In [ ]:
oportunidade = oportunidade_media_setor(df_aux, setor='Logística')
oportunidade.sort_values('DY (12M)Acumulado', ascending=False, inplace=True)
oportunidade